# Load Data From SQL

In [1]:
import pandas as pd
import configparser
import sqlalchemy
from sqlalchemy import create_engine

config = configparser.ConfigParser()
config.read(r'../config.ini')

['../config.ini']

In [ ]:
serverNames = config.get('Database', 'ServerName').split(',')
databaseNames = config.get('Database', 'DatabaseName').split(',')

connected = False

for i in range(max(len(serverNames), len(databaseNames))):
    try:
        print(f'Connection attempt to server "{serverNames[i].strip()}" database "{databaseNames[i].strip()}"...')
        engine = create_engine(f'mssql+pyodbc://{serverNames[i].strip()}/{databaseNames[i].strip()}?driver=SQL Server')
        connection = engine.connect()
        print('SQL server successfully connected')
        connected = True
        break
    except sqlalchemy.exc.OperationalError:
        print(f'SQL server failed to connect')

if not connected:
    raise RuntimeError("Could not connect to any SQL server")

Connection attempt to server "T2131-06\SQLEXPRESS" database "Practical_AI_amongus"...
SQL server failed to connect
Connection attempt to server "NOTEBOOK\SQLEXPRESS" database "Practical_AI_CA1"...
SQL server successfully connected


In [ ]:
cursor = connection.execute('SELECT * FROM orders')
df = pd.DataFrame(data=cursor.fetchall(), columns=cursor.keys())
connection.close()

In [ ]:
df.head()

# Exploratory Data Analysis